In [1]:
import pyomo.environ as pyo

# Define sets and indices
I = range(1, 18)  # Patients (1 to 17)
P = range(16)     # Proton fractions (0 to 15)

# Define parameters
C = 100  # Maximum proton fractions capacity

# Read BED scores from CSV file (assuming 'ProblemData.csv' is in the same directory)
#This is just one way of doing this, the data can be loaded in a couple ways.
BED = {}
with open('ProblemData.csv', 'r') as f:
    for i, line in enumerate(f):
        BED[i+1] = [float(x) for x in line.strip().split(',')]

# Create the Pyomo model
model = pyo.ConcreteModel()

# Define decision variables
model.x = pyo.Var(I, P, domain=pyo.Binary)

# Objective function (maximize total BED score)
model.obj = pyo.Objective(expr=sum(BED[i][p] * model.x[i, p] for i in I for p in P), sense=pyo.maximize)

# Constraint 1: Proton capacity constraint
model.capacity_constraint = pyo.Constraint(expr=sum(p * model.x[i, p] for i in I for p in P) <= C)

# Constraint 2: Treatment assignment constraint (each patient gets exactly one treatment)
model.assignment_constraint = pyo.ConstraintList()
for i in I:
    model.assignment_constraint.add(expr=sum(model.x[i, p] for p in P) == 1)

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can choose other solvers like gurobi, cplex, etc.
solver.solve(model)

# Print the results
print("Optimal Solution:")
for i in I:
    for p in P:
        if model.x[i, p].value > 0.5:
            print(f"Patient {i} receives {p} proton fractions (BED score: {BED[i][p]})")

print("\nTotal BED Score:", model.obj())

Optimal Solution:
Patient 1 receives 15 proton fractions (BED score: 0.44)
Patient 2 receives 8 proton fractions (BED score: 0.5)
Patient 3 receives 3 proton fractions (BED score: 0.75)
Patient 4 receives 0 proton fractions (BED score: 0.3)
Patient 5 receives 5 proton fractions (BED score: 0.26)
Patient 6 receives 0 proton fractions (BED score: 0.8)
Patient 7 receives 4 proton fractions (BED score: 0.67)
Patient 8 receives 13 proton fractions (BED score: 0.28)
Patient 9 receives 4 proton fractions (BED score: 0.22)
Patient 10 receives 5 proton fractions (BED score: 0.8)
Patient 11 receives 6 proton fractions (BED score: 0.53)
Patient 12 receives 0 proton fractions (BED score: 0.2)
Patient 13 receives 5 proton fractions (BED score: 0.75)
Patient 14 receives 0 proton fractions (BED score: 0.01)
Patient 15 receives 10 proton fractions (BED score: 0.41)
Patient 16 receives 10 proton fractions (BED score: 0.899999999999999)
Patient 17 receives 12 proton fractions (BED score: 0.42)

Total BE